# Zoho API orchestration

In [ ]:
import requests
import json
from IPython.display import JSON
from datetime import datetime
import pandas as pd
from difflib import get_close_matches

In [ ]:
with open('./data/self_client.json') as client_info, open('./data/portals_info.json') as portals_info, open('./data/refresh_tokens.json') as tokens_file:
    client_app_data = json.load(client_info)
    zoho_portals_data = json.load(portals_info)
    tokens_dict = json.load(tokens_file)

In [ ]:
TOKEN_URI = 'https://accounts.zoho.com/oauth/v2/token'
PROJECTS_API_URI = f'https://projectsapi.zoho.com/restapi/portal/{zoho_portals_data["portal_id"]}/projects/'
BOOKS_API_URI = 'https://www.zohoapis.com/books/v3'
CRM_APU_URI = 'https://www.zohoapis.com/crm/v4'

In [ ]:
zoho_apps_api_sessions = tokens_dict['tokens'].copy()

refresh_payload = {
    'grant_type':'refresh_token',
    'client_id':client_app_data['client_id'],
    'client_secret':client_app_data['client_secret']
}

for scope, refresh_token in tokens_dict['tokens'].items():
    refresh_payload['refresh_token'] = refresh_token
    refresh_response = requests.post(TOKEN_URI, data=refresh_payload)
    print(scope, refresh_response)
    
    # Create app specific session
    s = requests.session()
    s.headers = {
        'Authorization':f'Bearer {refresh_response.json()["access_token"]}'
    }
    # Store in sessions dict
    zoho_apps_api_sessions[scope] = s

# Requests

## POST

In [ ]:
create_project_uri = get_projects

In [ ]:
create_proj_params = {
    'name':'Test project - generated from Vittorio via API',
    'layout_id':'2072013000000571041'
}

In [ ]:
create_project_response = s.post(PROJECTS_API_URI + create_project_uri, data=create_proj_params)

In [ ]:
create_prj_resp = json.loads(create_project_response.content.decode())

In [ ]:
JSON(create_prj_resp)

## GET

### Flexible get request

In [ ]:
s = zoho_apps_api_sessions['projects_tasks_read']
get_req = PROJECTS_API_URI + '2072013000000405095/tasks/'

In [ ]:
get_req

In [ ]:
get_resp = s.get(get_req)

In [ ]:
get_resp.content

In [ ]:
JSON(get_resp.json())

In [ ]:
tasks_json = get_resp.json()

In [ ]:
for task in tasks_json['tasks']:
    print(task.get('completed'))

In [ ]:
JSON(tasks_json)

### Get Projects list

In [ ]:
s = zoho_apps_api_sessions['projects_all']
projects = []
records_per_page = 50
retrieved_projects = []
starting_round = True
i = 0

while((len(retrieved_projects) == records_per_page) | starting_round):
    starting_round = False
    retrieve_response = s.get(PROJECTS_API_URI, params={'index':i, 'range':records_per_page, 'status':'active'})
    print(retrieve_response)
    retrieved_projects = retrieve_response.json()['projects']
    projects.extend(retrieved_projects)
    i+=records_per_page

In [ ]:
JSON(projects)

In [ ]:
len(projects)

#### Manipulation from project list

In [ ]:
api_desc_snip = 'imported via API'

In [ ]:
api_projects = []
for pr in projects:
    if api_desc_snip in pr.get('description'):
        api_projects.append(pr.get('id'))

In [ ]:
projects[0]['name']

In [ ]:
webinar_ids = [prj['id'] for prj in projects if 'webinar' in prj['name'].lower()]

## Delete

In [ ]:
for to_delete_id in api_projects:
    delete_r = s.delete(PROJECTS_API_URI + f'/restapi/portal/{zoho_portals_data["portal_id"]}/projects/{to_delete_id}/')
    print(delete_r)

In [ ]:
delete_r.json()

## Update

In [ ]:
def update_uri(prj_id):
    return PROJECTS_API_URI + str(prj_id) + '/'

## add CRM account ID

In [ ]:
s = zoho_apps_api_sessions['crm_accounts_read']

In [ ]:
def get_accounts(page_token=None):
    params = {'fields':'Account_Name',
             'page_token':page_token}
    return s.get(CRM_APU_URI + '/Accounts', params=params)

In [ ]:
crm_accounts = []

get_crm_accounts_r = get_accounts()
crm_acc_r_content = get_crm_accounts_r.json()
crm_accounts.extend(crm_acc_r_content['data'])
next_page_token = crm_acc_r_content.get('info').get('next_page_token')

while(next_page_token):
    get_crm_accounts_r = get_accounts(next_page_token)
    crm_acc_r_content = get_crm_accounts_r.json()
    crm_accounts.extend(crm_acc_r_content['data'])
    next_page_token = crm_acc_r_content.get('next_page_token')

In [ ]:
crm_accs_df = pd.DataFrame.from_records(crm_accounts).set_index('Account_Name')

In [ ]:
for prj in projects:
    if prj['custom_fields'][0].get('Account')

In [ ]:
projects_df = pd.DataFrame([(prj['id'], prj['custom_fields'][0].get('Account')) for prj in projects], columns=['prj_id','acc_name'])

In [ ]:
projects_df['acc_id'] = projects_df.acc_name.apply(lambda x: crm_accs_df.loc[get_close_matches(x, crm_accs_df.index)[0]])

In [ ]:
s = zoho_apps_api_sessions['projects_all']
for i, row in projects_df.iterrows():
    r = s.post(update_uri(row.prj_id), data={
        'UDF_CHAR2':row.acc_id
    })
    print(r)
    time.sleep(1.5)

## training templates layouts

In [ ]:
trainig_projects = [proj['id'] for proj in response_dict['projects'] if 'training' in proj['name'].lower()]

In [ ]:
layout_id = '2072013000000571041'

In [ ]:
for prj_id in webinar_ids:
    update_template_r = s.post(update_uri(prj_id), data={'layout_id':layout_id})
    print(update_template_r)

In [ ]:
target_prj = '2072013000000480639'
update_template_r = s.post(update_uri(target_prj), data={'layout_id':layout_id})

In [ ]:
update_uri(target_prj)

In [ ]:
update_template_r.json()

## training types

In [ ]:
training_types = ['Triage', 'TECC', 'ECG', 'POCUS']

In [ ]:
getprojects_data = response_dict['projects']

In [ ]:
[(proj['name'], proj['id']) for proj in getprojects_data if any([train_type.lower() in proj['name'].lower() for train_type in ])]

In [ ]:
for proj in getprojects_data:
    proj_name_contains_training_type = any([training_type.lower() in proj['name'].lower() for training_type in training_types])
    if (proj_name_contains_training_type): print(proj['name'])

## archive old projects

In [ ]:
import time

In [ ]:
for prj_id in [prj['id'] for prj in projects]:
    update_r = s.post(update_uri(prj_id), data={'status':'active'})
    print(update_r)
    time.sleep(1)

# Import old projects

In [ ]:
import pandas as pd
import os
import sys
import numpy as np
sys.path.append(os.environ['pythonemkfdata'])
import dataload
import time

dataframes = dataload.load_emkf_data()

## Import old trainings

### Triage, tecc, ecg

In [ ]:
gsheet_trainings = dataframes['kd_google_drive']
gsheet_trainings.date = pd.to_datetime(gsheet_trainings.date, dayfirst=True)
non_zoho_trainings = gsheet_trainings.query('date < "2023-05-12"')

In [ ]:
crm_data = dataframes['crm_export'][['Account ID', 'Account Name', 'Account Type']]

In [ ]:
crm_hospitals = crm_data.query('`Account Type`.str.contains("Hospital|Hosptial")')

In [ ]:
non_zoho_trainings.assign(crm_account_name = non_zoho_trainings.facility_name.apply(lambda x: get_close_matches(x, crm_hospitals['Account Name'])[0]))

In [ ]:
trainings_with_crm_info = pd.merge(non_zoho_trainings, crm_hospitals, left_on='crm_account_name', right_on='Account Name')
trainings_with_crm_info_slim = trainings_with_crm_info.drop(['Account Name', 'Account Type'], axis=1)
trainings_with_crm_info_slim['Account ID'] = trainings_with_crm_info_slim['Account ID'].str.replace('zcrm_', '')

In [ ]:
trainings_dict = dict(zip([ttype.lower() for ttype in training_types], training_types))

In [ ]:
to_add_trainings = trainings_with_crm_info_slim

In [ ]:
to_add_trainings

### Webinars

In [ ]:
pd.to_datetime(dataframes['webinar_participants'].Date, dayfirst=True).nunique()

In [ ]:
webinar_df = dataframes['webinar_participants']
webinar_participants = webinar_df.groupby('Date')['Presenter - 1'].count().rename('num_participants').reset_index()

In [ ]:
webinar_participants['date'] = pd.to_datetime(webinar_participants.Date, dayfirst=True)

In [ ]:
for i, row in webinar_participants.iterrows():
    print(i)
    break

### Actual importing

In [ ]:
projects_session = zoho_apps_api_sessions['projects_all']

In [ ]:
for i, row in webinar_participants.iterrows():
    project_date = row.date.strftime('%m-%d-%Y')
    payload = {
        'name':'Webinar #' + str(i),
        'description':'This webinar has been imported via API by Vittorio Rossi',
        'layout_id':'2072013000000571041',
        'group_id':'2072013000000312007',
        'start_date':project_date,
        'end_date':project_date,
        'public':'yes',
        'UDF_CHAR1':'Knowledge and Development',
        'UDF_LONG1':row.num_participants,
        'UDF_CHAR3':'Webinar',
        # 'UDF_CHAR3':trainings_dict[get_close_matches(row.course_type.lower(), trainings_dict.keys())[0]],
        # 'UDF_TEXT1':f'{{"module_id":"{row["Account ID"]}","value":"{row.crm_account_name}"}}'
        'UDF_TEXT1':f'{{"module_id":"5621728000001081001","value":"Emergency Medicine Kenya Foundation"}}'
    }
    create_project_response = projects_session.post(PROJECTS_API_URI, data=payload)
    try:
        proj_json = create_project_response.json()['projects'][0]
        proj_id = proj_json['id']
        proj_name = proj_json['name']
        print(i, "Created proj successfully", proj_name)
        
        update_project_via_id_uri = PROJECTS_API_URI + str(proj_id) + '/'
        update_proj_response = projects_session.post(update_project_via_id_uri, data={'custom_status':'2072013000000020116'})
        time.sleep(.3)
    except KeyError:
        print('Failed to creat proj')

In [ ]:
JSON(create_project_response.json())